# Login and access data from AforroNet

This uses requests and lxml to access data from https://aforronet.igcp.pt/

The server does not send an intermediate certificate. I went to https://cryptoreport.websecurity.symantec.com/checker/#certChecker, pasted the site and downloaded the certificate.

Download the certificate and add it to lib/python3.4/site-packages/requests/cacert.pem or lib/python3.4/site-packages/certifi/cacert.pem (I just pasted it in there).

## Required imports

In [ ]:
import os
import requests
from lxml import html

import configparser
from datetime import datetime, date
import re
import pandas as pd
from bs4 import BeautifulSoup

from sqlalchemy import create_engine

### Variables with some needed values. Replace them as needed.

In [ ]:
config = configparser.ConfigParser()
config.read(os.path.join(os.path.expanduser('~'), '.config', 'me.ini'))


username = config['aforro']['username']
password = config['aforro']['password']
nif = config['aforro']['nif']

dburl = config['db']['url']
engine = create_engine(dburl)

# this is the position the mouse is in the "Entrar" button at the time the form is submited.
login_x = 0
login_y = 0

In [ ]:
headers = {
    'User-Agent': 'Mozilla/5.0',
    'Origin': 'https://aforronet.igcp.pt',
    'Referer': 'https://aforronet.igcp.pt/Iimf.AforroNet.UI/Services/Login/Login.aspx'
}


session = requests.Session()

r = session.get('https://aforronet.igcp.pt/', headers=headers)
tree = html.fromstring(r.text)

# nif elements requested for login.
req_nif = tree.xpath('//span[@id="LblInforma"]//strong')
first_pos = int(req_nif[0].text[0])
sec_pos = int(req_nif[1].text[0])

view_state = tree.xpath('//input[@name="__VIEWSTATE"]')[0].value

# cookies
cookies = requests.utils.cookiejar_from_dict(requests.utils.dict_from_cookiejar(session.cookies))

payload = {
    '__EVENTTARGET': "",
    '__EVENTARGUMENT': "",
    '__VIEWSTATE': view_state,
    'txtIdentificacao': username,
    'txtSenha': password,
    'dlPrim_Pos': int(nif[first_pos - 1]),
    'dlSeg_Pos': int(nif[sec_pos - 1]),
    'btLoginAfr.x': login_x,
    'btLoginAfr.y': login_y
}

In [ ]:
r = session.post('https://aforronet.igcp.pt/Iimf.AforroNet.UI/Services/Login/Login.aspx', data=payload, cookies=cookies, headers=headers)

tree = html.fromstring(r.text)
view_state = tree.xpath('//input[@name="__VIEWSTATE"]')[0].value

cookies = requests.utils.cookiejar_from_dict(requests.utils.dict_from_cookiejar(session.cookies))

payload = {
    '__EVENTTARGET': "lnkconsultaSaldos",
    '__EVENTARGUMENT': "",
    '__VIEWSTATE': view_state
}

In [ ]:
r = session.post('https://aforronet.igcp.pt/Iimf.AforroNet.UI/services/login/Menu.aspx', data=payload, cookies=cookies, headers=headers)

tables = BeautifulSoup(r.text, "lxml").find_all('table')
table = list(tables[12].stripped_strings)
# set . formating for numbers
table = "|".join(table).replace('.', '').replace(',', '.').replace(' - 1', '-1')
# remove authorized user info
table = "|".join(table.split('|', 7)[0:8:7])
# split into the different series
table = re.split('\|SubTotal\:\|[0-9.]{0,}\|?', table)[:-1]

value_date = date.today().replace(day=1)

final_values = []
for row in table:
    series, values = row.split('|', 1)
    final_values += [[series, value_date] + list(v) for v in zip(*[iter(values.split('|'))]*5)]

In [ ]:
aforro = pd.read_sql('aforro', engine)

if value_date not in aforro.value_date.dt.date.unique():
    new_values = pd.DataFrame(final_values, columns=aforro.columns)
    new_values.unit_value = new_values.unit_value.astype(float)
    new_values.value_date = pd.to_datetime(new_values.value_date)
    new_values.subscription_date = pd.to_datetime(new_values.subscription_date, format="%d-%m-%Y")
    new_values.to_sql('aforro', engine, if_exists='append', index=False)
else:
    print('Values already exist. skipping.')

In [ ]:
aforro = pd.read_sql('aforro', engine)

In [ ]:
aforro.to_sql('aforro', engine, if_exists='replace', index=False)